### 实现Modify Kneser-Ney Smoothing

In [283]:
import jieba
from collections import defaultdict
import time
from pyltp import SentenceSplitter
import numpy as np

In [256]:
FILE = open('../chinese_corpus.txt').read()

In [257]:
line = FILE.split('\n')

### 1.Counting

这一部分统计的是语料中unigram和bigram的原始计数

In [258]:
def get_unigram_count(corpus):
    '''
    统计分词后的语料中 每种ungram的个数
    '''
    unigram = defaultdict()
    i = 0
    for line in corpus:
        tokens = ['<s>'] + jieba.lcut(line) + ['</s>']
        for word in tokens:
            if word in unigram:
                unigram[word] += 1
            else:
                unigram[word] = 1
        i += 1
        if (i%1000) == 0 :
            print('处理进度{}%'.format((i/len(corpus))*100))
    return unigram

In [259]:
unigram = get_unigram_count(line)

处理进度51.92107995846313%


In [260]:
unigram

defaultdict(None,
            {'<s>': 1926,
             '此外': 11,
             '，': 3629,
             '自': 14,
             '本周': 1,
             '（': 75,
             '6': 119,
             '月': 171,
             '12': 23,
             '日': 125,
             '）': 72,
             '起': 16,
             '除': 4,
             '小米': 4,
             '手机': 65,
             '等': 159,
             '15': 9,
             '款': 9,
             '机型': 4,
             '外': 5,
             '其余': 3,
             '已': 43,
             '暂停': 4,
             '更新': 6,
             '发布': 23,
             '含': 3,
             '开发': 9,
             '版': 4,
             '/': 38,
             '体验版': 1,
             '内测': 1,
             '稳定版': 1,
             '暂不受': 1,
             '影响': 22,
             '以': 52,
             '确保': 6,
             '工程师': 5,
             '可以': 79,
             '集中': 6,
             '全部': 35,
             '精力': 3,
             '进行': 86,
             '系统优化': 1,
             '工作'

In [261]:
def get_bigram_count(corpus):
    '''
    统计分词后的语料中 每种bigram的个数 
    '''
    bigram = defaultdict()
    n = 0
    for line in corpus:
        tokens = ['<s>'] + jieba.lcut(line) + ['</s>']
        for i in range(len(tokens)-1):
            pre_word = tokens[i]
            word = tokens[i+1]
            bi = (pre_word,word)
            if bi in bigram:
                bigram[bi] += 1
            else:
                bigram[bi] = 1
        n += 1
        if (n%1000) == 0:
            print('处理进度{}%'.format((n/len(corpus))*100))
            
    return bigram

In [262]:
bigram = get_bigram_count(line)

处理进度51.92107995846313%


In [263]:
bigram

defaultdict(None,
            {('<s>', '此外'): 8,
             ('此外', '，'): 9,
             ('，', '自'): 5,
             ('自', '本周'): 1,
             ('本周', '（'): 1,
             ('（', '6'): 2,
             ('6', '月'): 87,
             ('月', '12'): 4,
             ('12', '日'): 3,
             ('日', '）'): 3,
             ('）', '起'): 2,
             ('起', '，'): 4,
             ('，', '除'): 3,
             ('除', '小米'): 1,
             ('小米', '手机'): 1,
             ('手机', '6'): 1,
             ('6', '等'): 1,
             ('等', '15'): 2,
             ('15', '款'): 1,
             ('款', '机型'): 1,
             ('机型', '外'): 1,
             ('外', '，'): 4,
             ('，', '其余'): 2,
             ('其余', '机型'): 1,
             ('机型', '已'): 1,
             ('已', '暂停'): 1,
             ('暂停', '更新'): 1,
             ('更新', '发布'): 1,
             ('发布', '（'): 1,
             ('（', '含'): 3,
             ('含', '开发'): 1,
             ('开发', '版'): 1,
             ('版', '/'): 1,
             ('/', '体验版'): 1,

### 2.Adjusting

在unigram中有些词可能出现的频数比较高，但是它作为接续词，也就是与它前面的一个词形成一个bigram的种类却比较少，这个时候就要做一定的调整\
应该给予这些词比较少的频数

In [225]:
def adjusting_pre_count(unigram,bigram):
    '''
    每个unigram与相邻的unigram组成一个bigram (word1,word2)
    统计所有可能与word2形成bigram的word1的种类数
    也就是语料中所有出现在word2前面的word的种类数
    <s> 前面不存在word 故用其本身的计算表示
    '''
    start = time.time()
    
    unigram_adjust = defaultdict()
    
    unigram_adjust['<s>'] = unigram['<s>']
    
    i = 0
    
    for word in unigram.keys():
        pre_word = []
        if word == '<s>':continue
        for bi in bigram.keys():
            if bi[-1] == word:
                if bi[0] in pre_word:
                    continue
                else:
                    pre_word.append(bi[0])
        pre_word_count = len(pre_word)
        
        unigram_adjust[word] = pre_word_count
        
        i += 1
        
        if (i%1000) == 0:
            print('目前进度{}'.format(i/len(unigram.keys())))
    
    end = time.time()
    
    print(end - start)
    
    return unigram_adjust

In [226]:
unigram_pre_count = adjusting_pre_count(unigram,bigram)

目前进度0.10037137408411122
目前进度0.20074274816822243
目前进度0.30111412225233364
目前进度0.40148549633644487
目前进度0.501856870420556
目前进度0.6022282445046673
目前进度0.7025996185887785
目前进度0.8029709926728897
目前进度0.9033423667570009
33.30477809906006


In [227]:
unigram_pre_count

defaultdict(None,
            {'<s>': 1926,
             '此外': 2,
             '，': 1918,
             '自': 6,
             '本周': 1,
             '（': 49,
             '6': 54,
             '月': 20,
             '12': 14,
             '日': 30,
             '）': 63,
             '起': 14,
             '除': 2,
             '小米': 3,
             '手机': 27,
             '等': 142,
             '15': 6,
             '款': 4,
             '机型': 3,
             '外': 5,
             '其余': 2,
             '已': 36,
             '暂停': 4,
             '更新': 4,
             '发布': 22,
             '含': 1,
             '开发': 5,
             '版': 4,
             '/': 37,
             '体验版': 1,
             '内测': 1,
             '稳定版': 1,
             '暂不受': 1,
             '影响': 14,
             '以': 34,
             '确保': 4,
             '工程师': 3,
             '可以': 52,
             '集中': 6,
             '全部': 16,
             '精力': 3,
             '进行': 73,
             '系统优化': 1,
             '工作': 36,

In [228]:
# def adjusting_next_count(unigram,bigram):
#     '''
#     每个unigram与相邻的unigram组成一个bigram (word1,word2)
#     统计所有可能与word1形成bigram的word2的种类数
#     也就是语料中所有出现在word1后面的word的种类数
#     </s> 后面不存在word 故不统计
#     '''
#     start = time.time()
    
#     unigram_adjust = defaultdict()
    
#     i = 0
    
#     for word in unigram.keys():
#         pre_word = []
#         if word == '</s>':continue
#         for bi in bigram.keys():
#             if bi[0] == word:
#                 if bi[-1] in pre_word:
#                     continue
#                 else:
#                     pre_word.append(bi[-1])
#         pre_word_count = len(pre_word)
        
#         unigram_adjust[word] = pre_word_count
        
#         i += 1
        
#         if (i%1000) == 0:
#             print('目前进度{}'.format(i/len(unigram.keys())))
    
#     end = time.time()
    
#     print(end - start)
    
#     return unigram_adjust

In [229]:
# unigram_next_count = adjusting_next_count(unigram,bigram)

In [230]:
# unigram_next_count

### 3.Discounting

$$ D_n(k) = k - \frac{(k+1)t_{n,1}t_{n,k+1}}{(t_{n,1}+2t_{n,2})t_{n,k}}$$

In [178]:
def get_tnk(ngram,k):
    count = []
    for key,value in ngram.items():
        if value == k :
            count.append(key)
    return len(count)

In [236]:
t11 = get_tnk(unigram_pre_count,1)
t12 = get_tnk(unigram_pre_count,2)
t13 = get_tnk(unigram_pre_count,3)
t14 = get_tnk(unigram_pre_count,4)
t21 = get_tnk(bigram,1)
t22 = get_tnk(bigram,2)
t23 = get_tnk(bigram,3)
t24 = get_tnk(bigram,4)

In [237]:
print(t11)
print(t12)
print(t13)
print(t14)
print(t21)
print(t22)
print(t23)
print(t24)

5899
1554
702
439
30764
3613
1028
439


In [238]:
def get_dnk(k,tn1,tn2,tnk,tnk_1):
    return k - (k+1)*tn1*tnk_1/((tn1+2*tn2)*tnk)

In [239]:
D11 = get_dnk(1,t11,t12,t11,t12)
D12 = get_dnk(2,t11,t12,t12,t13)
D13 = get_dnk(3,t11,t12,t13,t14)
D21 = get_dnk(1,t21,t22,t21,t22)
D22 = get_dnk(2,t21,t22,t22,t23)
D23 = get_dnk(3,t21,t22,t23,t24)

In [240]:
print(D11)
print(D12)
print(D13)
print(D21)
print(D22)
print(D23)

0.6549350505162651
1.1124239276787296
1.3617294177969208
0.8097920505396157
1.3087742364062622
1.616736536237777


### 4.Normalization

In [272]:
def pseudo_probability(word,n):
    if n == 1:
        if word not in unigram_pre_count:
            pro = 0
        else:
            uni_adjust_count = unigram_pre_count[word]
            if uni_adjust_count == 0 : 
                d1 = 0
            elif uni_adjust_count == 1:
                d1 = D11
            elif uni_adjust_count == 2:
                d1 = D12
            elif uni_adjust_count >= 3 :
                d1 = D13
            adjust_value_list = [value for key,value in unigram_pre_count.items() if key != '<s>']
            adjust_sum = sum(adjust_value_list)
            pro = (uni_adjust_count - d1) / adjust_sum
    if n == 2:
        if word not in bigram:
            pro = 0
        else:
            bi_count = bigram[word]
            if bi_count == 0:
                d2 = 0
            elif bi_count == 1:
                d2 = D21
            elif bi_count == 2:
                d2 = D22
            elif bi_count >= 3:
                d2 = D23
            pre_word_count = unigram[word[0]]
            pro = (bi_count - d2) / pre_word_count

    return pro

In [273]:
def back_off_weight(word):
    next_word_1 = [];next_word_2 = [];next_word_3 = []
    for key,value in bigram.items():
        if value == 1 and key[0] == word:
            next_word_1.append(key[-1])
        if value == 2 and key[0] == word :
            next_word_2.append(key[-1])
        if value == 3 and key[0] == word :
            next_word_3.append(key[-1])
          
    count1 = len(set(next_word_1))
    count2 = len(set(next_word_2))
    count3 = len(set(next_word_3))
    
    back_off = (D21*count1 + D22*count2 + D23*count3) / (unigram[word])
    
    return back_off

### 5.Interpolation

In [274]:
def uniform_distribution(word):
    
    u = pseudo_probability(word,1)
    vocabulary = len(unigram.keys())
    
    next_word_1 = [];next_word_2 = [];next_word_3 = []
    for key,value in unigram_pre_count.items():
        if value == 1:
            next_word_1.append(key[-1])
        if value == 2:
            next_word_2.append(key[-1])
        if value == 3:
            next_word_3.append(key[-1])
          
    count1 = len(set(next_word_1))
    count2 = len(set(next_word_2))
    count3 = len(set(next_word_3))
    
    uni_cnt = sum([value for key,value in unigram_pre_count.items() if key != '<s>'])
    b = (D11*count1+D12*count2+D13*count3) / uni_cnt
    
    return (u+b/vocabulary),b

In [275]:
def interpolation(words):
    u = pseudo_probability(words,2)
    
    if words[0] in unigram.keys():
        b = back_off_weight(words[0])
    else:
        b = uniform_distribution(words[0])[-1]
        
    nc = uniform_distribution(words[-1])[0]
    
    return u+b*nc

### 6.Test

In [280]:
def cut(sentence):
    tokens = ['<s>'] + jieba.lcut(sentence) + ['</s>']
    return tokens

In [284]:
def get_sentence_probability(sentence):
    tokens = cut(sentence)
    pro = np.log10(1)
    bigram_tokens = [(tokens[i],tokens[i+1]) for i in range(len(tokens)-1)]
    for bi in bigram_tokens:
        bi_pro = interpolation(bi)
        pro += np.log10(bi_pro)
    return pro

In [290]:
s1 = '白石麻衣天下第一'
s2 = '白石天下麻衣第一'
s3 = '白麻天第石衣下一'

In [291]:
print(get_sentence_probability(s1))
print(get_sentence_probability(s2))
print(get_sentence_probability(s3))

-22.020455831511075
-24.400258624380495
-25.381672533581405
